In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
! pip install playsound
from playsound import playsound


# Function to load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)
    return images, labels

# Load datasets
i_close_images, i_close_labels = load_images_from_folder('/content/drive/MyDrive/train/Closed', 0)
i_open_images, i_open_labels = load_images_from_folder('/content/drive/MyDrive/train/Open', 1)
yawn_images, yawn_labels = load_images_from_folder('/content/drive/MyDrive/train/yawn', 2)
not_yawn_images, not_yawn_labels = load_images_from_folder('/content/drive/MyDrive/train/no_yawn', 3)

# Combine datasets
images = np.array(i_close_images + i_open_images + yawn_images + not_yawn_images)
labels = np.array(i_close_labels + i_open_labels + yawn_labels + not_yawn_labels)

# One-hot encode the labels
labels = to_categorical(labels, num_classes=4)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Save the model
model.save('model.h5')



  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=ff012175caf93c2589e79fe97304dc921656ec58ac7984a9bccb953e8aee10b6
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


Epoch 1/10
74/74 [==============================] - 17s 205ms/step - loss: 11.3143 - accuracy: 0.6076 - val_loss: 0.5552 - val_accuracy: 0.7585
Epoch 2/10
74/74 [==============================] - 17s 231ms/step - loss: 0.5292 - accuracy: 0.8057 - val_loss: 0.4128 - val_accuracy: 0.8673
Epoch 3/10
74/74 [==============================] - 14s 196ms/step - loss: 0.4136 - accuracy: 0.8384 - val_loss: 0.2705 - val_accuracy: 0.8810
Epoch 4/10
74/74 [==============================] - 18s 246ms/step - loss: 0.3280 - accuracy: 0.8707 - val_loss: 0.2911 - val_accuracy: 0.8707
Epoch 5/10
74/74 [==============================] - 14s 188ms/step - loss: 0.2476 - accuracy: 0.9031 - val_loss: 0.3284 - val_accuracy: 0.8759
Epoch 6/10
74/74 [==============================] - 15s 199ms/step - loss: 0.2293 - accuracy: 0.9167 - val_loss: 0.2275 - val_accuracy: 0.8980
Epoch 7/10
74/74 [==============================] - 13s 180ms/step - loss: 0.1842 - accuracy: 0.9328 - val_loss: 0.2107 - val_accuracy: 0.906

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the trained model
model = tf.keras.models.load_model('model.h5')

# Labels for the predictions
labels_dict = {0: 'Closed Eyes', 1: 'Open Eyes', 2: 'Yawning', 3: 'Not Yawning'}

# Initialize webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Webcam successfully opened.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image")
            break

    # Preprocess the frame
    resized_frame = cv2.resize(frame, (64, 64))
    normalized_frame = resized_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Predict the class
    prediction = model.predict(input_frame)
    class_id = np.argmax(prediction)
    label = labels_dict[class_id]

    # Display the prediction
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Webcam', frame)

    # Play an alert sound if necessary
    if class_id == 0 or class_id == 2:  # If eyes are closed or yawning
        playsound('/content/alarm.mp3')
if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()
cv2.destroyAllWindows()
